In [1]:
import os
import shutil
import tensorflow as tf

gpu_devices = tf.config.list_physical_devices('GPU')
if gpu_devices:    
    print(tf.config.experimental.get_device_details(gpu_devices[0]))
    print()

{'device_name': 'NVIDIA GeForce GTX 1650', 'compute_capability': (7, 5)}



In [2]:
from pathlib import Path
import numpy as np

In [ ]:
IMAGE_SIZE = (299, 299)  # All images contained in this dataset are 299x299 (originally, to match Inception v3 input size).
SEED = 17

# Head directory containing all image subframes. Update with the relative path of your data directory.
colab_folder = "../data"
data_head_dir = Path(colab_folder + '/data')

# Find all subframe directories
subdirs = [Path(subdir.stem) for subdir in data_head_dir.iterdir() if subdir.is_dir()]
src_image_ids = ['_'.join(a_path.name.split('_')[:3]) for a_path in subdirs]

## Data Loading and Pre-Processing

In [ ]:
# Load train/val/test subframe IDs
def load_text_ids(file_path):
    """Simple helper to load all lines from a text file"""
    with open(file_path, 'r') as f:
        lines = [line.strip() for line in f.readlines()]
    return lines

# Load the subframe names for the three data subsets.
train_ids = load_text_ids('./train_source_images.txt')
validate_ids = load_text_ids('./val_source_images.txt')
test_ids = load_text_ids('./test_source_images.txt')

# Generate a list containing the dataset split for the matching subdirectory names.
subdir_splits = []
for src_id in src_image_ids:
    if src_id in train_ids:
        subdir_splits.append('train')
    elif src_id in validate_ids:
        subdir_splits.append('validate')
    elif(src_id in test_ids):
        subdir_splits.append('test')
    else:
        print(f'{src_id}: Did not find designated split in train/validate/test list.')
        subdir_splits.append(None)

In [ ]:
import random
from PIL import Image
import cv2

In [ ]:
# Update the batch and buffer size as per your model requirements
buffer_size = 64
batch_size = 8